In [1]:
# from __future__ import print_function
from numpy.random import seed

# TensorFlow and tf.keras
import tensorflow as tf
import keras

from DataGenerator import DataGenerator
from i3d_inception import *

# Helper libraries
seed(1)
import numpy as np
import matplotlib

matplotlib.use('Agg')
from matplotlib import pyplot as plt
import os
import h5py
import scipy.io as sio
import glob
import gc

from keras.models import load_model, Model, Sequential
from keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Activation, Dense, Dropout, ZeroPadding2D)
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, StratifiedShuffleSplit
from keras.layers.advanced_activations import ELU

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import keras.backend as K

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
# CHANGE THESE VARIABLES ---
data_folder = r'C:\Users\kentw\OneDrive - University of Toronto\PycharmProjects\kinetics-i3d\data\MAA' 
#mean_file = 'flow_mean.mat'  # Used as a normalisation for the input of the network

save_features = True  # Boolean flag if we save features in h5py
save_plots = True

# Set to 'True' if you want to restore a previous trained models
# Training is skipped and test is done
use_checkpoint = False # Set to True or False
# --------------------------

best_model_path = 'models/'
plots_folder = 'plots/'
checkpoint_path = 'models/fold_'

features_file = 'features_MAA.h5'
labels_file = 'labels_MAA.h5'
features_key = 'features'
labels_key = 'labels'

gpu_num = 1 
num_features = [None, 7, 1, 1, 1024]  # Specific dimension of features for 64-frame clips
batch_norm = False
learning_rate = 0.1
mini_batch_size = 4
weight_0 = 1.0  # A higher weight of 0 prioritizes learning in class 0
epochs = 100
dropout_prob=0.0

weights = 'rgb_imagenet_and_kinetics'
name = 'train_classifier'

# Name of the experiment
exp = 'slips_lr{}_batchs{}_batchnorm{}_w0_{}_{}_{}'.format(learning_rate,
                                                           mini_batch_size,
                                                           batch_norm,
                                                           weight_0, 
                                                           name,
                                                           weights)

# Input dimensions
NUM_FRAMES = 64
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_CLASSES = 2


In [3]:
# Functions 
def plot_training_info(case, metrics, save, history):
    """
    Function to create plots for train and validation loss and accuracy
    Input:
    * case: name for the plot, an 'accuracy.png' or 'loss.png' will be concatenated after the name.
    * metrics: list of metrics to store: 'loss' and/or 'accuracy'
    * save: boolean to store the plots or only show them.
    * history: History object returned by the Keras fit function.
    """
    plt.ioff()
    if 'accuracy' in metrics:
        fig = plt.figure()
        plt.plot(history['acc'])
        plt.plot(history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        if save:
            plt.savefig(case + 'accuracy.png')
            plt.gcf().clear()
        else:
            plt.show()
        plt.close(fig)

    # summarize history for loss
    if 'loss' in metrics:
        fig = plt.figure()
        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        # plt.ylim(1e-3, 1e-2)
        plt.yscale("log")
        plt.legend(['train', 'val'], loc='upper left')
        if save:
            plt.savefig(case + 'loss.png')
            plt.gcf().clear()
        else:
            plt.show()
        plt.close(fig)


def list_data():
    """
    Function to list existing data set by filename and label 
    """
    vids = {'pass': [], 'fail': []}
    
    for file in os.listdir(data_folder):
        if not file.lower().endswith('.npy'):
            continue
        else:
            file_dir = os.path.join(data_folder, file)
            ID = file_dir.split('\\')[-1].rstrip('.npy')
            # Classify
            result = ID.split('_')[2][1]
            if result == "P": vids["pass"].append(ID)
            else: vids["fail"].append(ID)
    return vids


def saveFeatures(feature_extractor,
                 features_file,
                 labels_file,
                 features_key,
                 labels_key):
    """
    Function to load the optical flow stacks, do a feed-forward through the feature extractor and store the
    output feature vectors in the file 'features_file' and the labels in 'labels_file'.
    Input:
    * feature_extractor: model without the top layer, which is the classifoer
    * features_file: path to the hdf5 file where the extracted features are going to be stored
    * labels_file: path to the hdf5 file where the labels of the features are going to be stored
    * features_key: name of the key for the hdf5 file to store the features
    * labels_key: name of the key for the hdf5 file to store the labels
    """

    # Fill the IDs and classes arrays
    classes = list_data()
    class0, class1 = classes['fail'], classes['pass']
    num_samples = len(class0) + len(class1)
    num_features[0] = num_samples
    
    # File to store the extracted features and datasets to store them
    # IMPORTANT NOTE: 'w' mode totally erases previous data
    # Write files in h5py format
    h5features = h5py.File(features_file, 'w')
    h5labels = h5py.File(labels_file, 'w')
    
    # Create data sets
    dataset_features = h5features.create_dataset(features_key,
                                                 shape=num_features,
                                                 dtype='float64')
    dataset_labels = h5labels.create_dataset(labels_key,
                                             shape=(num_samples, 1),
                                             dtype='float64')

    # Process class 0 
    gen = DataGenerator(class0, np.zeros(len(class0)), 1)
    for i in range(len(class0)):
                        
        rgb_images, rgb_labels = gen.__getitem__(i)
        predictions = feature_extractor.predict(rgb_images)

        dataset_features[i, :] = predictions
        dataset_labels[i, :] = 0
                
        del rgb_images, rgb_labels
        gc.collect()
        
    # Process class 1
    gen = DataGenerator(class1, np.ones(len(class1)), 1)
    for i in range(len(class0), num_samples):
        
        rgb_images, rgb_labels = gen.__getitem__(i)        
        prediction = feature_extractor.predict(rgb_images)

        dataset_features[i, :] = predictions
        dataset_labels[i, :]= 1

        del rgb_images, rgb_labels
        gc.collect()

    h5features.close()
    h5labels.close()


In [4]:
# trainer 
def main():
    # ========================================================================
    # I3D ARCHITECTURE
    # ========================================================================
    model = Inception_Inflated3d(
        include_top=False,
        weights='rgb_imagenet_and_kinetics',
        input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
        classes=NUM_CLASSES)
    
    # ========================================================================
    # FEATURE EXTRACTION
    # ========================================================================
    if save_features:
        saveFeatures(model, features_file,
                     labels_file, features_key,
                     labels_key)

    # ========================================================================
    # TRAINING
    # ========================================================================    
    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999,
                epsilon=1e-08)
    model.compile(optimizer=adam, loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    do_training = True
    compute_metrics = True
    threshold = 0.5

    if do_training:
        # Import data
        h5features = h5py.File(features_file, 'r')
        h5labels = h5py.File(labels_file, 'r')
        
        # X_full will contain all the feature vectors extracted
        X_full = h5features[features_key]
        _y_full = np.asarray(h5labels[labels_key])
        
        # Indices of 0 and 1 in the data set
        zeroes_full = np.asarray(np.where(_y_full == 0)[0])
        ones_full = np.asarray(np.where(_y_full == 1)[0])
        zeroes_full.sort()
        ones_full.sort()

        # Traditional Machine Learning methodology
        # Method get_n_splits() returns the number of splits 
        kf_0 = KFold(n_splits=5, shuffle=True)
        kf_0.get_n_splits(X_full[zeroes_full, ...])

        kf_1 = KFold(n_splits=5, shuffle=True)
        kf_1.get_n_splits(X_full[ones_full, ...])

        sensitivities = []
        specificities = []
        fars = []
        mdrs = []
        accuracies = []

        fold_number = 1
        
        # CROSS-VALIDATION: Stratified partition of the dataset into train/test sets
        for ((train_index_0, test_index_0),
             (train_index_1, test_index_1)) in zip(
            kf_0.split(X_full[zeroes_full, ...]),
            kf_1.split(X_full[ones_full, ...])
        ):
            
            train_index_0 = np.asarray(train_index_0)
            test_index_0 = np.asarray(test_index_0)
            train_index_1 = np.asarray(train_index_1)
            test_index_1 = np.asarray(test_index_1)

            # Train and Test Set
            X = np.concatenate((X_full[zeroes_full, ...][train_index_0, ...],
                                X_full[ones_full, ...][train_index_1, ...]))
            _y = np.concatenate((_y_full[zeroes_full, ...][train_index_0, ...],
                                 _y_full[ones_full, ...][train_index_1, ...]))
            X2 = np.concatenate((X_full[zeroes_full, ...][test_index_0, ...],
                                 X_full[ones_full, ...][test_index_1, ...]))
            _y2 = np.concatenate((_y_full[zeroes_full, ...][test_index_0, ...],
                                  _y_full[ones_full, ...][test_index_1, ...]))

            # Create a validation subset from the training set
            val_size = 0.2
            
            zeroes = np.asarray(np.where(_y == 0)[0])
            ones = np.asarray(np.where(_y == 1)[0])
            
            zeroes.sort()
            ones.sort()

            trainval_split_0 = StratifiedShuffleSplit(n_splits=1,
                                                      test_size=val_size / 2,
                                                      random_state=None)
            indices_0 = trainval_split_0.split(X[zeroes, ...],
                                               np.argmax(_y[zeroes, ...], 1))
            trainval_split_1 = StratifiedShuffleSplit(n_splits=1,
                                                      test_size=val_size / 2,
                                                      random_state=None)
            indices_1 = trainval_split_1.split(X[ones, ...],
                                               np.argmax(_y[ones, ...], 1))
            
            train_indices_0, val_indices_0 = indices_0.__next__()
            train_indices_1, val_indices_1 = indices_1.__next__()

            X_train = np.concatenate([X[zeroes, ...][train_indices_0, ...],
                                      X[ones, ...][train_indices_1, ...]], axis=0)
            y_train = np.concatenate([_y[zeroes, ...][train_indices_0, ...],
                                      _y[ones, ...][train_indices_1, ...]], axis=0)
            X_val = np.concatenate([X[zeroes, ...][val_indices_0, ...],
                                    X[ones, ...][val_indices_1, ...]], axis=0)
            y_val = np.concatenate([_y[zeroes, ...][val_indices_0, ...],
                                    _y[ones, ...][val_indices_1, ...]], axis=0)
           
            # ==================== CLASSIFIER ========================

            extracted_features = Input(shape=(7, 1, 1, 1024,), dtype='float32', name='input')

            # Batch size of 1 does not need normalization
            if batch_norm:
                x = BatchNormalization(axis=-1, momentum=0.99,
                                       epsilon=0.001)(extracted_features)

            # Classification block
            x = extracted_features
            # x = AveragePooling3D((2, 7, 7), strides=(1, 1, 1), padding='valid', name='global_avg_pool')(x)
            x = Dropout(dropout_prob)(x)

            x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
                    use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')

            num_frames_remaining = int(x.shape[1])
            x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

            # logits (raw scores for each class)
            x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
                       output_shape=lambda s: (s[0], s[2]))(x)

            # if not endpoint_logit
            x = Activation('softmax', name='prediction')(x)

            # Compile
            classifier = Model(input=extracted_features, output=x, name='classifier')
            fold_best_model_path = best_model_path + exp + '_MAA_fold_{}.h5'.format(fold_number)
            classifier.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


            if not use_checkpoint:
                # ==================== TRAINING ========================
                # weighting of each class: only the fall class gets
                # a different weight
                class_weight = {0: weight_0, 1: 1}

                # callback definition
                metric = 'val_loss'
                e = EarlyStopping(monitor=metric, min_delta=0, patience=20,
                                  mode='auto')
                c = ModelCheckpoint(fold_best_model_path, monitor=metric,
                                    save_best_only=True,
                                    save_weights_only=False, mode='auto')
                callbacks = [e, c]

                # Batch training
                if mini_batch_size == 0:
                    history = classifier.fit(X_train, y_train,
                                             validation_data=(X_val, y_val),
                                             batch_size=X_train.shape[0],
                                             nb_epoch=epochs,
                                             shuffle='batch',
                                             class_weight=class_weight,
                                             callbacks=callbacks)
                else:
                    history = classifier.fit(X_train, y_train,
                                             validation_data=(X_val, y_val),
                                             batch_size=mini_batch_size,
                                             nb_epoch=epochs,
                                             shuffle='batch',
                                             class_weight=class_weight,
                                             callbacks=callbacks)

                plot_training_info(plots_folder + exp, ['accuracy', 'loss'],
                                   save_plots, history.history)

                classifier = load_model(fold_best_model_path)

                # Use full training set (training+validation)
                X_train = np.concatenate((X_train, X_val), axis=0)
                y_train = np.concatenate((y_train, y_val), axis=0)

                if mini_batch_size == 0:
                    history = classifier.fit(X_train, y_train,
                                             batch_size=X_train.shape[0],
                                             nb_epoch=1,
                                             shuffle='batch',
                                             class_weight=class_weight)
                else:
                    history = classifier.fit(X_train, y_train,
                                             batch_size=mini_batch_size,
                                             nb_epoch=1,
                                             shuffle='batch',
                                             class_weight=class_weight)

                classifier.save(fold_best_model_path)

            # ==================== EVALUATION ========================
            # Load best model
            print('Model loaded from checkpoint')

            classifier = load_model(fold_best_model_path)


            if compute_metrics:
                predicted = classifier.predict(np.asarray(X2))
                predicted = predicted[:,1]

                # Array of predictions 0/1
                predicted = np.asarray(predicted).astype(int)
                
                # Compute metrics and print them
                cm = confusion_matrix(_y2, predicted, labels=[0, 1])
                tp = cm[0][0]
                fn = cm[0][1]
                fp = cm[1][0]
                tn = cm[1][1]
                tpr = tp / float(tp + fn)
                fpr = fp / float(fp + tn)
                fnr = fn / float(fn + tp)
                tnr = tn / float(tn + fp)
                precision = tp / float(tp + fp)
                recall = tp / float(tp + fn)
                specificity = tn / float(tn + fp)
                f1 = 2 * float(precision * recall) / float(precision + recall)
                accuracy = accuracy_score(_y2, predicted)

                print('\n')
                print('FOLD {} results:'.format(fold_number))
                print('TP: {}, TN: {}, FP: {}, FN: {}'.format(tp, tn, fp, fn))
                print('TPR: {}, TNR: {}, FPR: {}, FNR: {}'.format(
                    tpr, tnr, fpr, fnr))
                print('Sensitivity/Recall: {}'.format(recall))
                print('Specificity: {}'.format(specificity))
                print('Precision: {}'.format(precision))
                print('F1-measure: {}'.format(f1))
                print('Accuracy: {}'.format(accuracy))
                print('\n')

                fold_number += 1

                # Store the metrics for this epoch
                sensitivities.append(tp / float(tp + fn))
                specificities.append(tn / float(tn + fp))
                fars.append(fpr)
                mdrs.append(fnr)
                accuracies.append(accuracy)


    print('5-FOLD CROSS-VALIDATION RESULTS ===================')
    print("Sensitivity: %.2f (+/- %.2f)" % (np.mean(sensitivities), np.std(sensitivities)))
    print("Specificity: %.2f (+/- %.2f)" % (np.mean(specificities), np.std(specificities)))
    print("FAR: %.2f (+/- %.2f)" % (np.mean(fars), np.std(fars)))  # False alarm rates 
    print("MDR: %.2f (+/- %.2f)" % (np.mean(mdrs), np.std(mdrs)))  # Missed detection rates
    print("Accuracy: %.2f (+/- %.2f)" % (np.mean(accuracies), np.std(accuracies)))
    

In [5]:
if __name__ == '__main__':
    if not os.path.exists(best_model_path):
        os.makedirs(best_model_path)
    if not os.path.exists(plots_folder):
        os.makedirs(plots_folder)
    main()











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:145: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classifier", inputs=Tensor("in..., outputs=Tensor("pr...)`
c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:181: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 77 samples, validate on 10 samples
Epoch 1/100
77/77 [==============================] - 1s 13ms/step - loss: 8.2644 - acc: 0.4675 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/100
77/77 [==============================] - 0s 1ms/step - loss: 7.7451 - acc: 0.5195 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/100
77/77 [===================

c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:203: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1
87/87 [==============================] - 0s 4ms/step - loss: 7.7811 - acc: 0.5172
Model loaded from checkpoint


FOLD 1 results:
TP: 12, TN: 0, FP: 11, FN: 0
TPR: 1.0, TNR: 0.0, FPR: 1.0, FNR: 0.0
Sensitivity/Recall: 1.0
Specificity: 0.0
Precision: 0.5217391304347826
F1-measure: 0.6857142857142856
Accuracy: 0.5217391304347826




...

Epoch 1/1
89/89 [==============================] - 0s 4ms/step - loss: 7.7874 - acc: 0.5169
Model loaded from checkpoint


FOLD 4 results:
TP: 11, TN: 0, FP: 10, FN: 0
TPR: 1.0, TNR: 0.0, FPR: 1.0, FNR: 0.0
Sensitivity/Recall: 1.0
Specificity: 0.0
Precision: 0.5238095238095238
F1-measure: 0.6875000000000001
Accuracy: 0.5238095238095238




c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:145: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="classifier", inputs=Tensor("in..., outputs=Tensor("pr...)`
c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:181: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 79 samples, validate on 10 samples
Epoch 1/100
79/79 [==============================] - 1s 17ms/step - loss: 7.7531 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/100
79/79 [==============================] - 0s 1ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/100
79/79 [==============================] - 0s 2ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/100
79/79 [==============================] - 0s 1ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/100
79/79 [==============================] - 0s 1ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/100
79/79 [==============================] - 0s 1ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/100
79/79 [==============================] - 0s 1ms/step - loss: 7.7530 - acc: 0.5190 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/100
79/79 [===================

c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:203: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1
89/89 [==============================] - 0s 4ms/step - loss: 7.7874 - acc: 0.5169
Model loaded from checkpoint


FOLD 5 results:
TP: 11, TN: 0, FP: 10, FN: 0
TPR: 1.0, TNR: 0.0, FPR: 1.0, FNR: 0.0
Sensitivity/Recall: 1.0
Specificity: 0.0
Precision: 0.5238095238095238
F1-measure: 0.6875000000000001
Accuracy: 0.5238095238095238


5-FOLD CROSS-VALIDATION RESULTS ===================
Sensitivity: 0.80 (+/- 0.40)
Specificity: 0.20 (+/- 0.40)
FAR: 0.80 (+/- 0.40)
MDR: 0.20 (+/- 0.40)
Accuracy: 0.51 (+/- 0.02)


In [6]:
# Miscellaneous
def miscellaneous():
    model = Inception_Inflated3d(
            include_top=False,
            weights='rgb_imagenet_and_kinetics',
            input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
            classes=NUM_CLASSES)

    rgb_sample = np.load(r"C:\Users\kentw\OneDrive - University of Toronto\PycharmProjects\kinetics-i3d\data\MAA\idapt518_sub253_DF_10-18-15.npy")
    rgb_sample = np.expand_dims(rgb_sample, axis=0)
    rgb_logits = model.predict(rgb_sample)
    print(rgb_logits.shape)
    
# miscellaneous()
